<a href="https://colab.research.google.com/github/Xiao-Hongru/LLM/blob/main/VIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vision transformer

In [ ]:
import torch
import torch.nn as nn

class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, num_classes=1000, dim=768, depth=12, heads=12, mlp_dim=3072, dropout=0.1):
        super(VisionTransformer, self).__init__()
        self.patch_size = patch_size # 每个patch的长度和宽度为16个像素
        self.dim = dim # 每个token的向量维度为768（=3*16*16）
        self.num_patches = (img_size // patch_size) ** 2 # 计算patch的数量，//是向下取整
        self.patch_to_embedding = nn.Linear(patch_size * patch_size * 3, dim)# 每个Patch经过一个全连接层压缩成一定维度的向量

        # 定义cls token和位置编码
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))# 定义可学习 token，用于产生最终的特征向量，作为图片分类依据
        self.position_embeddings = nn.Parameter(torch.randn(1, self.num_patches + 1, dim))# 定义可学习的位置编码

        self.transformer = nn.Transformer(dim, heads, depth, dim_feedforward=mlp_dim, dropout=dropout)# 定义 Transformer 模型
        self.mlp_head = nn.Linear(dim, num_classes)# 定义encoder之后的MLP分类头

    def forward(self, x):
        """
        Forward pass of the Vision Transformer model.
        Args:
            x: Input tensor of shape (B, C, H, W).
        Returns:
            Tensor: Predicted logits of shape (B, num_classes).
        """
        # 在前向传播中，首先将输入图像分成多个patch，并将这些patch转换为适合Transformer输入的嵌入向量。
        B, C, H, W = x.shape # B是batch size，C是通道数，H是图片高度，W是图片宽度
        x = x.view(B, C, H // self.patch_size, self.patch_size, W // self.patch_size, self.patch_size)# 将图像划分为小patch块，结果形状为（B,C,num_patches_height,patch_size,num_patches_width,patch_size）
        x = x.permute(0, 2, 4, 3, 5, 1).contiguous()# 调整张量维度顺序为（B,num_patches_height,num_patches_width,patch_size,patch_size,C）
        # .contiguous 类似于深拷贝 .permute 转换维度
        x = x.view(B, self.num_patches, -1)# 将patches展平为一维向量
        x = self.patch_to_embedding(x)# 将展平的patch通过一个线性层转换为嵌入向量

        # 添加cls token，进行Positional Encoding，向量拼接
        cls_tokens = self.cls_token.expand(B, -1, -1)# 将分类标记 [CLS] token 从（1,1，dim）扩展到（B，1，dim），即为每个样本复制一个标记
        x = torch.cat((cls_tokens, x), dim=1)# 将cls_tokens与patches嵌入x，以第一个维度拼接，结果形状为（B，num_patches+1，dim）
        x += self.position_embeddings # 与位置编码直接相加，得到具有位置编码信息的特征向量

        #Transformer处理
        x = x.permute(1, 0, 2)  # 适配 Transformer 输入格式 (seq_len, batch, dim)
        x = self.transformer(x,x) # 将调整后的张量输入Transformer
        x = x[0]  # 取出 cls_token 的输出

        x = self.mlp_head(x) # 得到最终的特征输出
        return x

# 测试模型
model = VisionTransformer()
dummy_input = torch.randn(1, 3, 224, 224)
output = model(dummy_input)
print(output.shape)  # 输出应为 (1, num_classes)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


torch.Size([1, 1000])
